# Creating a custom dataset

The default datasets in this repo focus on tasks with relatively few states and/or actions, like the two-step task or reversal learning tasks.
In this notebook, we will walk through the creation of a new dataset class.

In [63]:
# Run in parent directory
import os
if os.getcwd().split('/')[-1] != 'tinyRNN':
    os.chdir('..')
assert os.getcwd().split('/')[-1] == 'tinyRNN'

In [64]:
import numpy as np

import sys
sys.path.append(os.getcwd() + '/simulating_experiments')
sys.path.append(os.getcwd() + '/plotting_experiments')

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating data

## Make a task

Here, we're examining a simple 5-armed Bernoulli bandit, where each arm's reward probability is sampled uniformly from $[0, 1]$. Having 5 actions is more than other tasks in this repo.

In [65]:
class MultiArmedBandit:
    '''5-armed Bernoulli bandit'''
    def __init__(self, n_actions=5):
        self.n_actions = n_actions

    def reset(self, n_trials=None):
        self.probs = np.random.uniform(0, 1, size=self.n_actions)

    def trial(self, choice):
        outcome = np.random.binomial(1, self.probs[choice])
        return (choice, outcome)

## Initialize MF agent

For our agent, we'll use a simple model-free RL agent.

In [66]:
from agents.MABCogAgent import MABCogAgent

mf = MABCogAgent(dict(cog_type='MF', n_actions=5))
mf.model.params[mf.model.param_names.index('iTemp')] = 2.

## Packaging data into a dataset

In [68]:
from simulating_experiments.simulate_experiment import simulate_exp
config = dict(
    n_blocks=500, n_trials=30, sim_seed=42, sim_exp_name='test', additional_name='',
    task='BartoloMonkey',
)
data = simulate_exp(mf, MultiArmedBandit(), config, save=False)

print(data['action'][0])
print(data['reward'][0])

Simulating agent MF with params [0.5, 2.0]
n_blocks 500 n_trials 30 sim_seed 42 sim_exp_name test additional_name 
[3 0 1 1 3 2 3 0 1 0 1 1 0 1 2 1 1 2 2 0 2 2 1 1 4 1 1 1 2 1]
[1 0 1 1 0 1 0 1 1 0 1 1 0 1 1 1 1 1 1 0 0 1 1 1 0 1 1 1 0 0]


We take just the essential data fields, for the purpose of this example. It's a bit clumsy, but one way to get this dataset loading to work is to first write out our data in its format.

In [69]:
import joblib
fn = 'files/SimpleDataset/5ab-mf.pkl'
joblib.dump({
    'action': data['action'],
    'reward': data['reward'],
}, fn)

['files/SimpleDataset/5ab-mf.pkl']

Now we can check to see whether the dataset loads.

In [82]:
x = np.zeros((3, 4, 5))

items = np.array([0, 2, 3], dtype=int)
x[0, items, 0] = 1
x

array([[[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.],
        [1., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]]])

In [90]:
from datasets.dataset_utils import Dataset
behav_data_spec = {
    'data': fn,
    'input_format': [
        dict(name='action', one_hot_classes=5),
        dict(name='reward'),
    ],
    'output_dim': 5, # number of output dimensions
    'target_name': 'action',
}
dd = Dataset('Simple', behav_data_spec=behav_data_spec)
dd.behav_to(dict(behav_format='tensor', output_h0=True))

print('actions are encoded properly')
print(data['action'][0][:10])
print(dd.torch_beahv_input[:, 0, :5][:10].max(1).indices)

print('rewards are encoded properly')
print(data['reward'][0][:10])
print(dd.torch_beahv_input[:, 0, 5][:10])

Total batch size: 500
actions are encoded properly
[3 0 1 1 3 2 3 0 1 0]
tensor([3, 0, 1, 1, 3, 2, 3, 0, 1, 0])
rewards are encoded properly
[1 0 1 1 0 1 0 1 1 0]
tensor([1., 0., 1., 1., 0., 1., 0., 1., 1., 0.], dtype=torch.float64)


# Training

Note: this will take a few minutes! But subsequent executions will load saved model.

In [93]:
from training_experiments.training import behavior_cv_training_config_combination

base_config = {
      ### dataset info
      'dataset': 'Simple',
      'behav_data_spec': behav_data_spec,

      ### model info
      'behav_format': 'tensor',
      'agent_type': 'RNN',
      'rnn_type': 'GRU', # which rnn layer to use
      'include_embedding': False,
      'input_dim': 6,
      'hidden_dim': 2, # dimension of this rnn layer
      'output_dim': 6, # dimension of action
      'output_h0': True, # whether initial hidden state included in loss
      'trainable_h0': False, # the agent's initial hidden state trainable or not
      'readout_FC': True, # whether the readout layer is full connected or not
      'one_hot': False, # whether the data input is one-hot or not

      'device': 'cpu',
      ### training info for one model
      'lr': 0.005,
      'l1_weight': 1e-5,
      'weight_decay': 0,
      'penalized_weight': 'rec',
      'max_epoch_num': 10000,
      'early_stop_counter': 200,
      'batch_size': 0, # no mini-batch
      ### training info for many models on dataset
      'outer_splits': 3,
      'inner_splits': 2,
      'single_inner_fold': True,
      'seed_num': 1,

      ### additional training info
      'save_model_pass': 'minimal', # 'full' for saving all results; 'minimal' for saving only the losses; 'none' for not saving results
      'training_diagnose': [], # can be a list of diagnose function strings

      ### current training exp path
      'exp_folder': '5ab_mf',
}

config_ranges = { # keys are used to generate model names
  'hidden_dim': [2,5],
}

behavior_cv_training_config_combination(base_config, config_ranges, n_jobs=1, verbose_level=1)

Already existing  D:\cognitive_dynamics\saved_model/5ab_mf/hidden_dim-2/allfold_summary.pkl
Already existing  D:\cognitive_dynamics\saved_model/5ab_mf/hidden_dim-5/allfold_summary.pkl


# Identify best models

For comparison, we also log the likelihood of the data under the generating model.

In [99]:
ll = np.mean(np.concat([
    row['trial_log_likelihood']
    for row in data['mid_vars']
]))
print('average log likelihood of generated data', ll)

average log likelihood of generated data -1.4120364328282298


In [96]:
from analyzing_experiments.analyzing_perf import find_best_models_for_exp

exp_folder = '5ab_mf'
find_best_models_for_exp(
    exp_folder, 'MABCog',
    additional_rnn_keys={'model_identifier_keys': ['block','distill','pretrained', 'distill_temp','teacher_prop',],},
    rnn_sort_keys=['block', 'hidden_dim'],
    has_rnn=True,
    has_cog=False,
    return_dim_est=True,
    include_acc=True,
    check_missing=False,
)


========Select best models based on inner_fold_perf_key: trainval_loss exp_folder: 5ab_mf
Searching for summary files...
Found 2 summary files.
Filtering by {'agent_type': 'RNN'}


100%|██████████| 2/2 [00:00<00:00, 875.64it/s]

5ab_mf/hidden_dim-2/outerfold0_innerfold0_seed0
Total batch size: 500
5ab_mf/hidden_dim-2/outerfold1_innerfold0_seed0
5ab_mf/hidden_dim-2/outerfold2_innerfold0_seed0
5ab_mf/hidden_dim-5/outerfold0_innerfold0_seed0
5ab_mf/hidden_dim-5/outerfold1_innerfold0_seed0
5ab_mf/hidden_dim-5/outerfold2_innerfold0_seed0
   block  test_loss  trainval_loss  train_loss  val_loss  test_acc  trainval_acc  train_acc   val_acc  hidden_dim rnn_type  readout_FC distill pretrained distill_temp teacher_prop agg_outer_fold                                                 agg_test_loss  total_test_loss  test_trial_num  total_trainval_loss  trainval_trial_num  total_train_loss  train_trial_num  total_val_loss  val_trial_num  test_loss_outer_std  test_loss_outer_sem  mean_train_trial_num  mean_val_trial_num                                                    agg_test_acc  total_test_acc  total_trainval_acc  total_train_acc  total_val_acc  test_acc_outer_std  test_acc_outer_sem  test_loss_mean_inner_sem  test_loss_


/Users/carlos/pd/tinyRNN/agents/RNNAgent.py:50: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(MODEL_SAVE_PATH / model_path / 'model.ckpt', map_locat